# OpenDC Reproducibility Capsule

# Topology Generation

In [2]:
import ipywidgets as widgets
from IPython.display import display
import os

from src.template_loader import *
from src.customization import *
from src.runner import *
from src.exporter import *

def clean_selection(selections, full_options):
    if not selections:
        return None

    if "[Select All]" in selections:
        return [opt for opt in full_options if opt not in ("[Keep original]", "[Select All]")]

    filtered = [s for s in selections if s not in ("[Keep original]", "[Select All]")]
    return filtered if filtered else None                            # Put it in utils later

# ---------------Topology Widgets-------------------------------------------------------------------------------------
topology_selector = widgets.Dropdown(options=os.listdir("topologies"), description='Topology:')
topology_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Topology')
topology_row = widgets.HBox([topology_selector, topology_upload])

# ---------------Carbon Widgets-------------------------------------------------------------------------------------
carbon_selector = widgets.SelectMultiple(
    options=['[Keep original]'] + ['[Select All]'] + os.listdir("carbon_traces"),
    description='Carbon:'
)
carbon_upload = widgets.FileUpload(accept='.parquet', multiple=False, description='Upload Carbon Traces')
carbon_row = widgets.HBox([carbon_selector, carbon_upload]) 

# ------------------ NoH --------------------------------------------------------------------------------------------
NoH_input = widgets.Text(
    value='1',
    placeholder='Enter number, list or range (e.g. 1, 2, 3 or 1-10:1)',
    description='# Hosts:',
    disabled=False
)

# -----------------Battery Values -----------------------------------------------------------------------------------
battery_capacity_input = widgets.Text(
    value='100',
    placeholder='Enter number, list or range (e.g. 100, 200 or 100-300:50)',
    description='Battery Capacity:',
    disabled=False
)

starting_CI_input = widgets.Text(
    value='0.1',
    placeholder='Enter number, list or range (e.g. 0.1, 0.2)',
    description='Start CI:',
    disabled=False
)

charging_speed_input = widgets.Text(
    value='1000',
    placeholder='Enter number, list or range (e.g. 1000, )',
    description='Charging Speed:',
    disabled=False
)

add_battery_checkbox = widgets.Checkbox(
    value=False,
    description='Include Battery',
    disabled=False
)

battery_row = widgets.VBox([
    battery_capacity_input,
    starting_CI_input,
    charging_speed_input,
])

generate_topology_button = widgets.Button(description="Generate Topologies")

output = widgets.Output()

def on_generate_topology_clicked(b):
    with output:
        output.clear_output()

        if topology_upload.value:
            topology_file = save_uploaded_file(topology_upload, "topologies")
            topology_value = topology_file
        else:
            topology_value = topology_selector.value

        if carbon_upload.value:
            carbon_file = save_uploaded_file(carbon_upload, "carbon_traces")
            carbon_values = carbon_file
        else:
            carbon_values = carbon_selector.value


        NoH_values = parse_input(NoH_input.value) or [1]
        battery_capacity_values = parse_input(battery_capacity_input.value) or [0]
        starting_CI_values = parse_input(starting_CI_input.value) or [0]
        charging_speed_values = parse_input(charging_speed_input.value) or [1000]
        
        include_battery = add_battery_checkbox.value
        
        update_topology_carbon_battery(
            topology_value,
            carbon_values,
            NoH_values,
            battery_capacity_values,
            starting_CI_values,
            charging_speed_values,
            include_battery
        )

generate_topology_button.on_click(on_generate_topology_clicked)

display(
    widgets.VBox([
        widgets.HTML("<b>OpenDC Topology Configurator</b>"),
        topology_row,
        carbon_row,
        NoH_input,
        add_battery_checkbox,
        battery_row,
        generate_topology_button,
        output
    ])
)
# Have a way to change parameters for one cluster and also add clusters, accept range, list, single values (think maybe for now just for single cluster and then change the same for all available)
# Add for convenience a call to a function to do it as well if widgets are inconvenient
# Ask Dante if I should ignore missing values or keep the one already existent
# Add ways to change CPU count, memory size, etc, maybe could be another customization section where it could be


In [ ]:
#If widgets are inconvenient, then you can call a function below with your parameters provided, an example call is like this (inputs need to be provided by yourself):
# for NoH in NoHList:
#  for carbon in carbon_list:
#     for capacity in capacities:
#         update_topology_carbon_battery(
        #     topology_value,
        #     carbon_values,
        #     NoH_values,
        #     battery_capacity_values,
        #     starting_CI_values,
        #     charging_speed_values,
        #     include_battery=True
        # )

# Experiment Generation

In [ ]:
# Generate different experiments, see what values to change, etc

# File Selector

In [3]:
experiment_queue = []

topology_selector = widgets.SelectMultiple(
    options= ['[Keep original]'] +  ['[Select All]'] + os.listdir("topologies"),
    description='Topologies:'
)

workload_selector = widgets.SelectMultiple(
    options=['[Keep original]'] + ['[Select All]'] + os.listdir("workload_traces"),
    description='Workloads:'
)

failure_selector = widgets.SelectMultiple(
    options=['[Keep original]'] + ['[Select All]'] + os.listdir("failure_traces"),
    description='Failures:'
)



name_selector = widgets.Text( value='custom_experiment.json', placeholder='Enter experiment name', description='Name:', disabled=False)

remove_selector = widgets.Dropdown(options=[], description='Remove From Queue:')
remove_button = widgets.Button(description="Remove Selected From The Queue")
remove_row = widgets.HBox([remove_selector, remove_button])

# topology_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("topologies"), description='Topology:')
topology_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Topology')
topology_row = widgets.HBox([topology_selector, topology_upload])

# workload_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("workload_traces"), description='Workload:')
workload_upload = widgets.FileUpload(multiple=False, description='Upload Workload')
workload_row = widgets.HBox([workload_selector, workload_upload])

# failure_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("failure_traces"), description='Failures:')
failure_upload = widgets.FileUpload(accept='.parquet', multiple=False, description='Upload Failures')
failure_row = widgets.HBox([failure_selector, failure_upload]) 

experiment_selector = widgets.Dropdown(options=os.listdir("experiments"), description='Experiment:')
experiment_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Experiment')
experiment_row = widgets.HBox([experiment_selector, experiment_upload])

add_experiment_button = widgets.Button(description="Add Experiment")
run_all_button = widgets.Button(description="Run All Experiments")
export_button = widgets.Button(description="Export the capsule as ZIP")
button_row = widgets.HBox([add_experiment_button, run_all_button, export_button])

output = widgets.Output()

def on_add_clicked(b): 
    with output:
        output.clear_output()

        if experiment_upload.value:
            experiment_file = save_uploaded_file(experiment_upload, "experiments")[0]
            refresh_dropdown(experiment_selector, "experiments", experiment_file, False)
        else:
            experiment_file = experiment_selector.value

        experiment_path = os.path.join("experiments", experiment_file)
        experiment = load_experiment_template(experiment_path)

        if experiment is None:
            return

        if topology_upload.value:
            topology_file = save_uploaded_file(topology_upload, "topologies")
            topology_values = topology_file
        else:
            topology_values = topology_selector.value

        if workload_upload.value:
            workload_file = save_uploaded_file(workload_upload, "workload_traces")
            workload_values = workload_file
        else:
            workload_values = workload_selector.value  #TO-DO FIX WORKLOAD FORMAT UPLOAD

        if failure_upload.value:
            failure_file = save_uploaded_file(failure_upload, "failure_traces")
            failure_values = failure_file
        else:
            failure_values = failure_selector.value


        experiment_name = name_selector.value

        print(f"Name: {experiment_name}")

        selections = {
            "name": experiment_name,
            "topology": clean_selection(topology_values, os.listdir("topologies")),
            "workload": clean_selection(workload_values, os.listdir("workload_traces")),
            "failures": clean_selection(failure_values, os.listdir("failure_traces")),
        }
        

        updated_experiment = update_experiment_fields(experiment, selections)

        filename = experiment_name
        save_json_file(updated_experiment, filename, "experiments")
        
        experiment_queue.append(selections)
        remove_selector.options = [exp["name"] for exp in experiment_queue]
        print(f"Added experiment '{experiment_name}' to the queue. Total queued: {len(experiment_queue)}")


def on_run_all_clicked(b):

    with output:
        output.clear_output()
        run_all_experiments(experiment_queue)

def on_remove_clicked(b):
    with output:
        output.clear_output()
        to_remove = remove_selector.value
        if to_remove:
            removed = False
            for i, exp in enumerate(experiment_queue):
                if exp["name"] == to_remove:
                    del experiment_queue[i]
                    removed = True
                    break
            if removed:
                print(f"Removed experiment {to_remove} from queue.")
                remove_selector.options = [exp["name"] for exp in experiment_queue]
            else:
                print(f"Experiment {to_remove} not found in queue.")
        else:
            print("No experiment selected to remove.")

def on_export_called(b):
    create_reproducibility_zip(experiment_queue)
    print("Capsule created")

add_experiment_button.on_click(on_add_clicked)
run_all_button.on_click(on_run_all_clicked)
remove_button.on_click(on_remove_clicked)
export_button.on_click(on_export_called)

display(
    widgets.VBox([
        widgets.HTML("<b>OpenDC Experiment Configurator</b>"),
        name_selector,
        experiment_row,
        topology_row,
        workload_row,
        failure_row,
        button_row,
        remove_row,
        output
    ])
)


# Think if include zip or other (Docker, tar, etc), add also output files
# Generate readme explaining what is where also with self reflection
# Make a file to run in case the user does not have notebook
# Ask about carbon and why my new runner still fails